<a href="https://colab.research.google.com/github/gundaminpde/2022/blob/main/forword.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

 

##### 파이썬으로 구현한 Forward Propagation Code

#### 함부로 실행하면 후회함.

############# 출처:인공지능 100점을 위한 파이썬 수학 – 임성국, BJPUBLiC

 

 

 

#############################

# MNIST 데이터 입력
#############################

 

 


import numpy as np
from keras.datasets import mnist
(x_train, t_train), (x_test, t_test) = mnist.load_data()
t_trainlbl, t_testlbl = t_train, t_test

# 28X28 을 784 로 수정
x_train = x_train.reshape(60000,784)    # 주석 (1)
x_test = x_test.reshape(10000,784)   

# one-hot label
T0 = np.zeros((t_train.size, 10))    #(60000,10) = 000
T1 = np.zeros((t_test.size, 10))    #(10000,10) = 000

for idx in range(t_train.size): T0[idx][t_train[idx]] = 1    #(3))
for idx in range(t_test.size): T1[idx][t_test[idx]] = 1

t_train, t_test = T0, T1

# normalize 0.0 ~ 1.0
x_train = x_train / 255
x_test = x_test / 255

print('MNIST DataSets 준비 완료')

 

 

#################################

#  함수정의

###################################

# 미분함수
def numerical_diff(f, x):
    h = 1e-4    # 0.0001
    nd_coef = np.zeros_like(x)
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        index = it.multi_index
        tmp = x[index]
        x[index] = tmp + h
        fxh2 = f()    # f(x+h)
        x[index] = tmp - h
        fxh1 = f()    # f(x-h)
        nd_coef[index] = (fxh2 - fxh1) / (2*h)
        x[index] = tmp
        it.iternext()
    return nd_coef

# 시그모이드
def sigmoid(x):
    return 1 / (1+np.exp(-x))

# 소프트맥스
def softmax(x):
    if x.ndim == 1:  # 기본 1개 처리과정 , 벡터입력
        x = x - np.max(x)
        return np.exp(x) / np.sum(np.exp(x))
    if x.ndim == 2:  # 배치용 n 개 처리, 행렬입력
        x = x.T - np.max(x.T, axis=0)
        return (np.exp(x) / np.sum(np.exp(x), axis=0)).T

# 크로스엔트로피오차
def cee(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)  # 크기가 1xN 인 2차원 행렬로 재구성
        y = y.reshape(1, y.size)
    result = -np.sum(t * np.log(y + 1e-7))  / y.shape[0]
    return result 

 

 

 

 

############################ 

##   프로세스별클래스생성

###########################

class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        result = x.copy()
        result[self.mask] = 0
        return result


class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        self.out = sigmoid(x)
        return self.out


class Affine:
    def __init__(self, W, b):
        self.W = W    # W0, W1
        self.b = b    # b0, b1
        self.x = None
        self.dW = None    # W0, W1 의 기울기
        self.db = None    # b0, b1 의 기울기

    def forward(self, x):
        self.x = x
        result = np.dot(self.x, self.W) + self.b
        return result


class SoftmaxWithLoss:
    def __init__(self):
        self.y = None    # 출력(계산결과)
        self.t = None    # 정답(MNIST레이블)
       
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        result = cee(self.y, self.t)
        return result

 

###########################

# 네트워크클래스생성

#######################3

class SimpleNetwork:
    def __init__(self, inputx, hidden, outy, weight):
        # 가중치 초기화
        self.netMat = {}
        self.netMat['W0'] = weight * np.random.randn(inputx, hidden)
        self.netMat['b0'] = np.zeros(hidden)
        self.netMat['W1'] = weight * np.random.randn(hidden, outy)
        self.netMat['b1'] = np.zeros(outy)

        # 계층 생성
        self.netLayers = {}
        self.netLayers['Affine1'] = Affine(self.netMat['W0'],
                                           self.netMat['b0'])
        self.netLayers['Relu1'] = Relu()
        self.netLayers['Affine2'] = Affine(self.netMat['W1'],
                                           self.netMat['b1'])
        self.netLayers['Softmax'] = SoftmaxWithLoss()

    def predict(self, x):
        x = self.netLayers['Affine1'].forward(x)
        x = self.netLayers['Relu1'].forward(x)
        x = self.netLayers['Affine2'].forward(x)
        return x
       
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.netLayers['Softmax'].forward(y, t)
   
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
       
    def numerical_gradient(self, x, t):
        lossfunc = lambda : self.loss(x, t)
        grads = {}
        grads['W0'] = numerical_diff(lossfunc, self.netMat['W0'])
        grads['b0'] = numerical_diff(lossfunc, self.netMat['b0'])
        grads['W1'] = numerical_diff(lossfunc, self.netMat['W1'])
        grads['b1'] = numerical_diff(lossfunc, self.netMat['b1'])
        return grads

 

 

 

##################################

# 미분을이용한학습과검증

#################################

import time    # Using Time Module(시간측정)
t1 = time.time()    # save nowTime(현재 시간 측정)

train_size = x_train.shape[0]    # size of TrainData (입력데이터 크기) 60000
lr = 0.1    # learning rate(학습률)
iter = 0    # Iternation Number (반복횟수)

iters_num = 10
batch_size = 20
iter_per_epoch = 1

network = SimpleNetwork(inputx=784, hidden=50, outy=10, weight = 0.2)

print('loss = _______  time = ________  n = ______ | [TrainAcc] [TestAcc]')

for i in range(iters_num):    # 10
    batch_mask = np.random.choice(train_size, batch_size)    #(1)주석
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
   
    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분(편 미분) 방식

    # 갱신
    for key in ('W0', 'b0', 'W1', 'b1'): network.netMat[key] -= lr * grad[key]
   
    loss = network.loss(x_batch, t_batch)

    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    iter = iter + 1

    print('loss = {:7.4f}  '.format(loss), end='')
    print('time = {:8.4f}  '.format(time.time()-t1), end='')   
    print('n = {:06d} |  {:6.4f}  {:9.4f}'.format(iter, train_acc, test_acc))

 

MNIST DataSets 준비 완료
loss = _______  time = ________  n = ______ | [TrainAcc] [TestAcc]
loss =  2.7990  time =  30.5042  n = 000001 |  0.1417     0.1461
loss =  2.1151  time =  59.6543  n = 000002 |  0.1262     0.1258
loss =  2.0780  time =  89.8155  n = 000003 |  0.1827     0.1862
loss =  1.9628  time = 120.3245  n = 000004 |  0.1939     0.1999
loss =  1.9029  time = 151.0767  n = 000005 |  0.2326     0.2403
loss =  1.6285  time = 182.0165  n = 000006 |  0.2487     0.2541
loss =  1.9789  time = 212.5545  n = 000007 |  0.2675     0.2708
loss =  1.3084  time = 243.1781  n = 000008 |  0.2996     0.3019
loss =  1.1188  time = 273.6971  n = 000009 |  0.3140     0.3179
loss =  1.6079  time = 303.8991  n = 000010 |  0.3209     0.3238
